Importing Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Getting the information of Cannada, from the wikipedia url

In [2]:
cannada_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
cannada = requests.get(cannada_url).text

In [3]:
soup = BeautifulSoup(cannada, 'xml')

Finding the table from the text

In [4]:
table =soup.find('table')

Creating a dataframe with the required columns

In [5]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

In [6]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        toronto.loc[len(toronto)] = row_data

In [7]:
toronto.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Removing the "Not Assigned" in the "Borough" column

In [8]:
toronto=toronto[toronto['Borough']!='Not assigned']
toronto.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Join the rows where PostalCode is same and Neighborhood is different

In [9]:
temp_toronto=toronto.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_toronto=temp_toronto.reset_index(drop=False)

Renaming the column name of Neighborhood

In [10]:
temp_toronto.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [11]:
temp_toronto.head()

,Postalcode,Neighborhood_joined
0,M1B,"Malvern, Rouge"
1,M1C,"Rouge Hill, Port Union, Highland Creek"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [12]:
toronto.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Merging the two dataframes for updated information

In [13]:
toronto_merge = pd.merge(toronto, temp_toronto, on='Postalcode')

In [14]:
toronto_merge.head()

,Postalcode,Borough,Neighborhood,Neighborhood_joined
0,M3A,North York,Parkwoods,Parkwoods
1,M4A,North York,Victoria Village,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights","Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","Queen's Park, Ontario Provincial Government"


Dropping the extra column, Neighborhood

In [15]:
toronto_merge.drop(['Neighborhood'],axis=1,inplace=True)

Dropping the duplicates.

In [16]:
toronto_merge.drop_duplicates(inplace=True)

Renaming the column the 'Neighborhood_joined' as 'Neighborhood'

In [17]:
toronto_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [18]:
toronto_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Shape of the final Dataframe

In [19]:
toronto_merge.shape

(103, 3)

Defining A new Function to get the Coordinates, with Postal Code

In [20]:

def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, cannada, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

Reading a file that have coordinates for Postal Codes

In [21]:
geocode=pd.read_csv('http://cocl.us/Geospatial_data')

In [22]:
geocode.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the Cannada Dataframe with the Geocode Dataframe, to get Coordinates for all the postal codes in Cannada

In [23]:
geocode.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geocode, toronto_merge, on='Postalcode')

In [24]:
geo_merged

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
...,...,...,...,...,...
98,M9N,43.706876,-79.518188,York,Weston
99,M9P,43.696319,-79.532242,Etobicoke,Westmount
100,M9R,43.688905,-79.554724,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,43.739416,-79.588437,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [25]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

In [26]:
geo_data.head()


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
